# IEEE-CIS Fraud Detection - data analisys

    Detailed infomation about competition:
https://www.kaggle.com/c/ieee-fraud-detection/overview

    Data analitycs for the competition dataset
    Done before modelling for features investigation and finding logical correlations

Importing libs, ignoring all warnings, defining plots style and color palette

In [1]:
import sys
import os
import wget
import zipfile
import string
import warnings
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes\
                    , metrics, svm, decomposition, ensemble
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#from keras import layers, models, optimizers
import xgboost 


warnings.simplefilter('ignore')
__location__ = sys.path[0]

Using TensorFlow backend.


    Data import

Train datasets:
- train_identity.csv -- shape(144233, 41) -- additional dataset, connected with transactions by TransactionID
- train_transaction.csv -- shape(590540, 394) -- main dataset with transactional history

Test datasets:
- test_identity.csv -- shape(141907, 41)
- test_transaction.csv -- shape(506691, 393)

Finals:
- sample_submission.csv -- submission finals file

In [2]:
train_identity = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/train_transaction.csv')
test_identity = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/test_identity.csv')
test_transaction = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/test_transaction.csv')
ss = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/sample_submission.csv')

In [4]:
non_num_cols = [train_transaction.columns[n] for n,i in enumerate(train_transaction.dtypes)
                if i not in ('int64','float64')]
#train_transaction[non_num_cols]

In [5]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()

for i in non_num_cols:
    train_transaction['mod_'+i] = encoder.fit_transform(train_transaction[i].fillna('0'))
    test_transaction['mod_'+i] = encoder.fit_transform(test_transaction[i].fillna('0'))

In [28]:
col_list = [x for x in train_transaction.columns if x not in non_num_cols+['isFraud']]

num = train_transaction[col_list]._get_numeric_data()
print(num[num < 0])# = 0 

#x_train, y_train, x_valid, y_valid = model_selection.train_test_split(train_transaction[col_list].fillna(0)
#                                                                      , train_transaction['isFraud'])

#print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

        TransactionID  TransactionDT  TransactionAmt  card1  card2  card3  \
0                 NaN            NaN             NaN    NaN    NaN    NaN   
1                 NaN            NaN             NaN    NaN    NaN    NaN   
2                 NaN            NaN             NaN    NaN    NaN    NaN   
3                 NaN            NaN             NaN    NaN    NaN    NaN   
4                 NaN            NaN             NaN    NaN    NaN    NaN   
5                 NaN            NaN             NaN    NaN    NaN    NaN   
6                 NaN            NaN             NaN    NaN    NaN    NaN   
7                 NaN            NaN             NaN    NaN    NaN    NaN   
8                 NaN            NaN             NaN    NaN    NaN    NaN   
9                 NaN            NaN             NaN    NaN    NaN    NaN   
10                NaN            NaN             NaN    NaN    NaN    NaN   
11                NaN            NaN             NaN    NaN    NaN    NaN   

In [12]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [27]:
# Naive Bayes
accuracy = train_model(naive_bayes.MultinomialNB(), x_train, x_valid, y_train)
print ("NB, Count Vectors: ", accuracy)

# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), x_train, x_valid, y_train)
print ("LR, Count Vectors: ", accuracy)

# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), x_train, x_valid, y_train)
print ("RF, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), x_train, x_valid, y_train)
print ("Xgb, Count Vectors: ", accuracy)

ValueError: Input X must be non-negative

In [ ]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram

In [ ]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], 
                                       trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)